## Rotating a batch of 3D paralleloids and then normalizing the orientation with the pytorch version

In [1]:
import torch
from plot_field import plot_scalar_field_torch, plot_inertia_torch
from rotation import get_rotation_tensor, rotate_torch, get_grids_torch
from inertia import barycenter_field_torch, get_rotation_params_torch, normalize_orientation_torch
%matplotlib widget

Two paralleloids represented by a scalar field are generated. <br>
Consider changing the gpu number according to your configuration.

In [2]:
rect1 = torch.tensor([(i,j,k) for i in range(10,45) for j in range(10,25) for k in range(10,15)]).long().to("cuda:0")
rect2 = torch.tensor([(i,j,k) for i in range(40,45) for j in range(5,25) for k in range(5,15)]).long().to("cuda:0")
tensor0 = torch.zeros((2,1,50,50,50))
for a in rect1:
    tensor0[0][0][a[0], a[1], a[2]] = 1.
for a in rect2:
    tensor0[1][0][a[0], a[1], a[2]] = 1.
grid_x, grid_y, grid_z = get_grids_torch(tensor0, "cuda:0")
barycenter_tab = [barycenter_field_torch(tensor0[i][0].to("cuda:0"), grid_x, grid_y, grid_z) for i in range(2)]
print("barycenters =", barycenter_tab)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
plot_scalar_field_torch(tensor0)

Plotting rotation vectors (axes)

In [ ]:
plot_inertia_torch(tensor0.to("cuda:0"))

A random angle and a random normal vector are generated

In [ ]:
angle = [torch.rand(1)*torch.pi for _ in range(2)]
normal = [torch.rand(3) for _ in range(2)]
for i in range(2):
    normal[i] /= torch.norm(normal[i])
print("angle = {}\nnormal vector = {}".format(angle, normal))

Rotate the paralleloid around its barycenter, with angle and normal vector computed above.<br>
Discretization process leads to small differences in the barycenter which in theory is rotaton-invariant.

In [ ]:
rotation_tensor_tab = [get_rotation_tensor(normal[i], angle[i]) for i in range(2)]
rect1 = rect1.float()
rect2 = rect2.float()
batch_rotated = rotate_torch(rotation_tensor_tab, tensor0, barycenter_tab)
plot_scalar_field_torch(batch_rotated)
print("barycenter after rotation : {}".format(barycenter_field_torch(batch_rotated, grid_x, grid_y, grid_z)))

In [ ]:
plot_inertia_torch(batch_rotated)

Align rotations axes with orthonormal basis

In [ ]:
batch_aligned = normalize_orientation_torch(batch_rotated)
plot_inertia_torch(batch_aligned)